# SHA-256 Hash Python Implementation
U.S. Department of Commerce. (2015). Secure Hash Standard (SHS). Available at: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf [Accessed 15 February 2020].

## High level process

1. Append padding bits ( '1' + (n * '0') + (len(input) in binary))
2. Message schedule 
3. Main loop


### Import and Script Operation Variables

In [1]:
import math, sys

In [2]:
debug = 0

### General Support Functions

In [3]:
# https://stackoverflow.com/questions/7396849/convert-binary-to-ascii-and-vice-versa
def text_to_bits(text, encoding='ascii', errors='surrogatepass'):
    bits = bin(int.from_bytes(text.encode(encoding, errors), 'big'))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def bits_to_text(bits, encoding='ascii', errors='surrogatepass'):
    n = int(bits, 2)
    return n.to_bytes((n.bit_length() + 7) // 8, 'big').decode(encoding, errors) or '\0'

In [11]:
def rotate(input_bits, direction, n_rotate):
    # Rotate = Circular left shift 2 bits
    if debug >= 3: print("[DEBUG]\tStarting function rotate")
    rotate = (n_rotate*-1) if direction == 'right' else n_rotate
    
    rotated_bits = input_bits[rotate:] + input_bits[:rotate]
    if debug >= 3: print("\tInput:",input_bits,"Rotated output:",rotated_bits)
        
    return rotated_bits

def shift(input_bits, direction, n_shift):
    # Rotate = Circular left shift 2 bits
    if debug >= 3: print("[DEBUG]\tStarting function shift")

    shifted_bits = None
    if direction == 'left':
        shifted_bits = input_bits[n_shift:] + ("0"*n_shift)
    elif direction == 'right':
        shifted_bits = ("0"*n_shift) + input_bits[:(len(input_bits)-n_shift)]

    if debug >= 3: print("\tInput:",input_bits,"Shifted output:",shifted_bits)
        
    return shifted_bits


def add_binary(s1, s2):
    # https://stackoverflow.com/questions/21420447/need-help-in-adding-binary-numbers-in-python
    if not s1 or not s2:
        return ''

    maxlen = max(len(s1), len(s2))

    s1 = s1.zfill(maxlen)
    s2 = s2.zfill(maxlen)

    result  = ''
    carry   = 0

    i = maxlen - 1
    while(i >= 0):
        s = int(s1[i]) + int(s2[i])
        if s == 2: #1+1
            if carry == 0:
                carry = 1
                result = "%s%s" % (result, '0')
            else:
                result = "%s%s" % (result, '1')
        elif s == 1: # 1+0
            if carry == 1:
                result = "%s%s" % (result, '0')
            else:
                result = "%s%s" % (result, '1')
        else: # 0+0
            if carry == 1:
                result = "%s%s" % (result, '1')
                carry = 0   
            else:
                result = "%s%s" % (result, '0') 

        i = i - 1;

    if carry>0:
        result = "%s%s" % (result, '1')
    return result[::-1]

# bitwize and
def bit_and(input1,input2):
    
    result = ""
    for index in range(len(input1)):
        if input1[index] == '1' and input2[index] == '1': 
            result += '1'
        else:
            result += '0'
    return result

# bitwize complement
def bit_complement(input1):
    
    result = ""
    for index in range(len(input1)):
        if input1[index] == '1': 
            result += '0'
        else:
            result += '1'
    return result

In [12]:

def split_into_blocks(input_str, block_length):
    blocks = [input_str[i:i+block_length] for i in range(0, len(input_str), block_length)]
    return blocks

def convert_binary_to_hex(binarydigits):
    hexdigits = '%0*X' % ((len(binarydigits) + 3) // 4, int(binarydigits, 2))
    return hexdigits

def convert_hex_to_binary(hexdigits):
    binarydigits = ""
    for hexdigit in hexdigits:
        binarydigits += bin(int(hexdigit,16))[2:].zfill(4)
    return binarydigits

def convert_hex_to_ascii(hex_input):
    return ''.join([chr(int(''.join(c), 16)) for c in zip(hex_input[0::2],hex_input[1::2])])


# xor
def xor(input1,input2):
    # xor
    # Example: xor(input1,input2)
    
    result = ""
    for index in range(len(input1)):
        if input1[index] == input2[index]: 
            result += '0'
        else:
            result += '1'
    return result

## Hash generation

In [13]:
def sha256(input_str):

    # initial hash value
    H = ['6a09e667', 'bb67ae85', '3c6ef372','a54ff53a', '510e527f', '9b05688c','1f83d9ab','5be0cd19']

    # constants
    K = ['428a2f98','71374491','b5c0fbcf','e9b5dba5','3956c25b','59f111f1','923f82a4','ab1c5ed5',
         'd807aa98','12835b01','243185be','550c7dc3','72be5d74','80deb1fe','9bdc06a7','c19bf174',
         'e49b69c1','efbe4786','0fc19dc6','240ca1cc','2de92c6f','4a7484aa','5cb0a9dc','76f988da',
         '983e5152','a831c66d','b00327c8','bf597fc7','c6e00bf3','d5a79147','06ca6351','14292967',
         '27b70a85','2e1b2138','4d2c6dfc','53380d13','650a7354','766a0abb','81c2c92e','92722c85',
         'a2bfe8a1','a81a664b','c24b8b70','c76c51a3','d192e819','d6990624','f40e3585','106aa070',
         '19a4c116','1e376c08','2748774c','34b0bcb5','391c0cb3','4ed8aa4a','5b9cca4f','682e6ff3',
         '748f82ee','78a5636f','84c87814','8cc70208','90befffa','a4506ceb','bef9a3f7','c67178f2']

    # Convert ascii to bits
    input_in_bits = text_to_bits(input_str)

    # Work out how many bits required for the binary expression of the length of the input string
    bin_length = math.ceil(math.log(len(input_in_bits))/math.log(2))

    # Calculate the number of 64 bit blocks that are required for the bin_length
    bit_blocks = math.ceil(bin_length/64)*64

    # Decimal length in binary
    length_in_binary = int(bin(len(input_in_bits))[2:])

    length_64bit_block = str(length_in_binary).rjust(bit_blocks, '0')

    zeros = "0" * (512-((len(length_64bit_block)+1+len(input_in_bits))%512))

    padded_message = str(input_in_bits) + "1" + str(zeros) + str(length_64bit_block)

    message_chunks = split_into_blocks(padded_message,512)

    for chunk in message_chunks:

        W = split_into_blocks(chunk,32)

        # Extend w to form message schedule array
        for i in range(16, 64):
            # +̃ is addition with truncation of the result to 32-bit.
            # https://crypto.stackexchange.com/questions/57763/in-a-very-simplistic-and-step-by-step-example-how-do-i-get-the-w-values-for-s

            # Useful sudo code from: https://en.wikipedia.org/wiki/SHA-2#SHA-256_.28a_SHA-2_variant.29_pseudocode
            s0 = xor(xor(rotate(W[i-15],'right',7),rotate(W[i-15],'right',18)),shift(W[i-15],'right',3))
            s1 = xor(xor(rotate(W[i-2],'right',17),rotate(W[i-2],'right',19)),shift(W[i-2],'right',10))
            temp_w = add_binary(add_binary(add_binary(W[i-16],s0),W[i-7]),s1)
            W.append(temp_w[-32:])

        # Initialise working variables
        a = convert_hex_to_binary(H[0])
        b = convert_hex_to_binary(H[1])
        c = convert_hex_to_binary(H[2])
        d = convert_hex_to_binary(H[3])
        e = convert_hex_to_binary(H[4])
        f = convert_hex_to_binary(H[5])
        g = convert_hex_to_binary(H[6])
        h = convert_hex_to_binary(H[7])

        # Main loop
        for i in range(64):
            S1 = xor(xor(rotate(e,'right',6), rotate(e,'right',11)),rotate(e,'right',25))
            ch = xor(bit_and(e,f),bit_and(bit_complement(e),g))
            temp1 = add_binary(add_binary(add_binary(add_binary(h,S1),ch),convert_hex_to_binary(K[i])),W[i])[-32:]
            S0 = xor(xor(rotate(a,'right',2), rotate(a,'right',13)),rotate(a,'right',22))
            maj = xor(xor(bit_and(a,b),bit_and(a,c)),bit_and(b,c))
            temp2 = add_binary(S0, maj)[-32:]
            h = g
            g = f
            f = e
            e = add_binary(d, temp1)[-32:]
            d = c
            c = b
            b = a
            a = add_binary(temp1, temp2)[-32:]

        H[0] = convert_binary_to_hex(add_binary(convert_hex_to_binary(H[0]),a)[-32:])
        H[1] = convert_binary_to_hex(add_binary(convert_hex_to_binary(H[1]),b)[-32:])
        H[2] = convert_binary_to_hex(add_binary(convert_hex_to_binary(H[2]),c)[-32:])
        H[3] = convert_binary_to_hex(add_binary(convert_hex_to_binary(H[3]),d)[-32:])
        H[4] = convert_binary_to_hex(add_binary(convert_hex_to_binary(H[4]),e)[-32:])
        H[5] = convert_binary_to_hex(add_binary(convert_hex_to_binary(H[5]),f)[-32:])
        H[6] = convert_binary_to_hex(add_binary(convert_hex_to_binary(H[6]),g)[-32:])
        H[7] = convert_binary_to_hex(add_binary(convert_hex_to_binary(H[7]),h)[-32:])

    digest = H[0] + H[1] + H[2] + H[3] + H[4] + H[5] + H[6] + H[7]

    return digest

In [14]:
input_text = "abcdefghijklmnopqrstuvwxyz123456789abcdefghijklmnopqrstuvwxyz123456789abcdefghijklmnopqrstuvwxyz123456789abcdefghijklmnopqrstuvwxyz123456789"

sha256(input_text)

# target
#ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
#ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad

#fd33e4a1d689e02a6ad3af24cc1b9a32cbc85ead80920bd9646d5c4352455de2
#FD33E4A1D689E02A6AD3AF24CC1B9A32CBC85EAD80920BD9646D5C4352455DE2

'FD33E4A1D689E02A6AD3AF24CC1B9A32CBC85EAD80920BD9646D5C4352455DE2'